 # UDACITY - VEHICLE DETECTION PROJECT



#### Hiep Truong Cong

### Content
* Feature extraction
    + Color feature extraction
    + Historgram fearture extraction
    + HOG feature extraction
    + Combination of extracted features
* Training a classifier
    + Prepare training data
    + Normalize the traing data
    + Training the classifier
    + Evaluate the classifier
* Sliding window search
    + Sliding window
    + Classifier each window
    + Combine detected windows
* Video pipline